In [34]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import meshio
import pyvista as pv
import torch

In [2]:
# df = pd.read_csv("../../data/MLgSA/stenosis_metadata.csv", sep=';')

In [40]:
from torch_geometric.data import Data

In [2]:
file_name = "../../data/MlgSA/../../data/MlgSA/data_patient_k_002_left_wss.vtu"


#reader = vtk.vtkXMLUnstructuredGridReader()
#reader.SetFileName(file_name)
#reader.Update()  # Needed because of GetScalarRange
#output = reader.GetOutput()
#wss = output.GetPointData().GetArray("longitudinal")

In [3]:
reader = pv.get_reader(file_name)
mesh = reader.read()

In [4]:
mesh

UnstructuredGrid (0x2f1fc1f32e0)
  N Cells:    84036
  N Points:   42225
  X Bounds:   2.608e+01, 6.523e+01
  Y Bounds:   -3.127e+01, -2.813e+00
  Z Bounds:   -1.154e+02, -4.099e+01
  N Arrays:   4

In [12]:
p = mesh.celltypes
np.unique(p)

array([5], dtype=uint8)

In [14]:
mesh.n_points

42225

In [26]:
def get_cells(mesh):
    """Returns a list of the cells from this mesh.
    This properly unpacks the VTK cells array.
    There are many ways to do this, but this is
    safe when dealing with mixed cell types."""
    ctype = 3
    shape = (mesh.n_cells, ctype+1)
    return mesh.cells.reshape(shape)[:, 1:]


def find_adjacent_cells(cells, index):
    """Given the cells array, find other cells that
    share vertices with the cell at ``index``"""
    cell = cells[index]
    idxs = []
    adjacent = []
    for i, c in enumerate(cells):
        if any([v in cell for v in c]):
            idxs.append(i)
            adjacent.append(c)
    return idxs, adjacent

In [8]:
nodes = mesh.points
cells = mesh.cells.reshape(-1, 4)[:, 1:]

In [10]:
num_nodes = nodes.shape[0]
adj = np.zeros((num_nodes, num_nodes), dtype=int)
for cell in cells:
    adj[cell[0], cell[1]] = 1
    adj[cell[1], cell[0]] = 1
    adj[cell[1], cell[2]] = 1
    adj[cell[2], cell[1]] = 1
    adj[cell[2], cell[0]] = 1
    adj[cell[0], cell[2]] = 1

In [12]:
%%time
adj = np.maximum(adj, adj.T)

CPU times: total: 20 s
Wall time: 25.2 s


In [19]:
cols = mesh.array_names

In [36]:
wss = np.asarray(mesh.point_data[cols[0]])

In [35]:
adj = torch.tensor(adj,dtype=torch.float)

In [37]:
node_features = torch.tensor(wss, dtype=torch.float)

In [38]:
edge_indices = adj.nonzero().t()

In [20]:


# Work flow

## Load data
## Assert cell type
## Isolate cell to populate adjacency
## Extract relevant data
## Store to new dataset

In [41]:
data = Data(x=node_features, edge_index=edge_indices)

In [47]:
torch.save(data, '../export/data.pt')

In [48]:
print(data)

Data(x=[42225], edge_index=[2, 252532])


In [1]:
edge_indices.shape

NameError: name 'edge_indices' is not defined